### **Data Preparation for Image Classification**

We'll demonstrate how to prepare image data for a classification task using Python and Pandas. Specifically, we'll:

1.   **Load image data from CSV files.**
2.   **Split the data into train, validation, and test sets.**
3.   **Organize the data into directories suitable for training a machine learning model.**


Let's get started by loading the necessary libraries and reading the CSV files containing image paths and corresponding labels. We'll then split the data and organize it into appropriate directories. Finally, we'll copy the images to their respective directories, readying them for model training.




In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import pandas as pd
import csv
import shutil
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Reading the train.csv and test.csv
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
print("Data loaded successfully.")
print(f"Train shape: {train.shape}, Test shape: {test.shape}")
print("Label distribution in train data:", train['label'].value_counts())

# Paths
train_dir = "./dataset/train"
test_dir = "./dataset/test"
csv_file = './dataset/train.csv'
output_dir = './dataset/'

# Read CSV file and prepare data list
data = []
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    for row in reader:
        image_path, class_name = row
        data.append((train_dir + '/' + image_path, class_name))
print("Data list prepared from CSV.")

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.1, random_state=42)
print("Data split into train, validation, and test sets.")

# Create output directories for the datasets
for dataset, data_items in [('train', train_data), ('val', valid_data), ('test', test_data)]:
    dataset_dir = os.path.join(output_dir, dataset)
    os.makedirs(dataset_dir, exist_ok=True)
    for _, class_name in data_items:
        class_dir = os.path.join(dataset_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
    print(f"Created directories for {dataset} dataset.")

# Copy images to respective class folders in train, validation, and test sets
for dataset, data_items in [('train', train_data), ('val', valid_data), ('test', test_data)]:
    for image_path, class_name in data_items:
        source_path = image_path
        destination_path = os.path.join(output_dir, dataset, class_name, os.path.basename(image_path))
        shutil.copy(source_path, destination_path)
    print(f"Images copied to {dataset} dataset directory.")

### **Training and Evaluating YOLO Model on Custom Dataset**

This code snippet outlines the process of training and evaluating a YOLO (You Only Look Once) model on a custom dataset for object detection tasks. YOLO is a popular deep learning algorithm known for its real-time object detection capabilities.

In [ ]:
# Import YOLO model
from ultralytics import YOLO

# Define class dictionary
class_dict = {0: 'Badminton', 1: 'Cricket', 2: 'Karate', 3: 'Soccer', 4: 'Swimming', 5: 'Tennis', 6: 'Wrestling'}

# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n-cls.pt')  # Load a pre-trained model

# Fine-tune the model on a custom dataset
result = model.train(data='./dataset/', epochs=1, imgsz=64)

# Evaluate the model on the test dataset
# eval_result = model.eval(data='G:/Disk D/NUCES Fast/Semester 6/AI/Final Project/code/dataset/test')

### **Visualizing Training Results in Colab**

We'll demonstrate how to visualize training results, specifically loss and validation accuracy, using matplotlib in Google Colab.

We first load the training results from a CSV file, typically generated during model training. Then, we plot the training loss and validation loss against epochs to understand how the model's performance evolves over time. Additionally, we plot the validation accuracy against epochs to observe the model's learning progress.

These visualizations provide insights into the training process, helping to assess the model's performance and identify potential areas for improvement. By analyzing these plots, researchers and practitioners can make informed decisions about model optimization and tuning strategies.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt


results_path = './runs/classify/train9/results.csv'

results = pd.read_csv(results_path)

plt.figure()
plt.plot(results['                  epoch'], results['             train/loss'], label='train loss')
plt.plot(results['                  epoch'], results['               val/loss'], label='val loss', c='red')
plt.grid()
plt.title('Loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()


plt.figure()
plt.plot(results['                  epoch'], results['  metrics/accuracy_top1'] * 100)
plt.grid()
plt.title('Validation accuracy vs epochs')
plt.ylabel('accuracy (%)')
plt.xlabel('epochs')

plt.show()


### **Testing at my end**

The provided code snippet demonstrates how to use the Ultralytics library to perform object detection using a custom YOLO (You Only Look Once) model.

1. **Importing Libraries:**
   - The `ultralytics` library is imported to utilize its YOLO object detection capabilities.
   - `torch` and `numpy` libraries are imported for general tensor and array manipulation.

2. **Loading the Model:**
   - The YOLO model is loaded using the `YOLO` class from the Ultralytics library. The path to the custom model weights (`last.pt`) is provided as an argument.

3. **Performing Object Detection:**
   - The `model` object is called with an image path (`'./pexels-case-originals-3800517.jpg'`) to perform object detection on that image.
   - The results of the detection are stored in the `results` variable.

4. **Extracting Prediction Results:**
   - The `results` object contains detection results such as bounding boxes, labels, and confidence scores.
   - The `names_dict` variable holds a dictionary mapping class indices to class names.
   - The `probs` variable stores the confidence scores for each detected object class.
   - The class with the highest confidence score is determined using `np.argmax(probs)` and printed out.

This code snippet provides a basic example of how to use a custom YOLO model for object detection and extract prediction results from the model's output.

In [ ]:
from ultralytics import YOLO
import torch

import numpy as np


model = YOLO('./runs/classify/train9/weights/last.pt')  # load a custom model

# model.eval()

results = model('./pexels-case-originals-3800517.jpg')  # predict on an image

# print(results)

names_dict = results[0].names

probs = results[0].probs.data.tolist()

print(names_dict)
print(probs)

print(names_dict[np.argmax(probs)])

### **Object Detection API Backend with Flask and Ultralytics YOLO**

This code sets up a Flask backend for an object detection API using the Ultralytics YOLO model. The purpose is to create a RESTful API endpoint (`/predict`) that accepts POST requests containing images, processes them using the YOLO model, and returns the predicted class based on the highest confidence score.

The backend is built using Flask, a lightweight web application framework for Python, and leverages the Ultralytics YOLO model for object detection tasks. Additionally, the Flask-CORS extension is used to enable cross-origin resource sharing, allowing requests from different origins.

The `/predict` endpoint handles incoming image files, checks their format, converts them to the required RGB format if necessary, and then performs object detection using the YOLO model. Finally, it extracts the prediction results and returns the predicted class along with its confidence score as a JSON response.

This backend provides a simple yet powerful solution for integrating object detection capabilities into web applications, allowing developers to easily deploy and serve machine learning models for real-world applications.

In [ ]:
from flask import Flask, request, jsonify
from ultralytics import YOLO
from PIL import Image
from flask_cors import CORS
import io

app = Flask(__name__)
CORS(app)
model = YOLO('./runs/classify/train9/weights/last.pt')

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'})

    file = request.files['image']
    image_bytes = file.read()

    try:
        # Open image using PIL to check format and convert to supported format
        image = Image.open(io.BytesIO(image_bytes))

        # Convert image to RGB format (if not already in RGB)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Process the image with the Ultralytics model
        results = model(image)

        # Extract prediction results and return
        names_dict = results[0].names
        probs = results[0].probs.data.tolist()
        predicted_class = names_dict[probs.index(max(probs))]

        return jsonify({'predicted_class': predicted_class})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)
